<a href="https://colab.research.google.com/github/Latamila/Latamila/blob/main/artesanalGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Lab 2</font>
## <font color='blue'>Construindo Seu Próprio ChatGPT Para Geração de Texto com PyTorch</font>

![DSA](imagens/Lab2.png)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [8]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00


In [3]:
!pip install -q transformers==4.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.3 MB/s eta 0:00:00


In [4]:
!pip install -q torch==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resol

In [5]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [6]:
# Importação das bibliotecas necessárias
import torch
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

transformers: 4.28.1
torch       : 2.0.0



In [22]:
# Função para geração de texto
def gera_texto(prompt, model, tokenizer, device, max_length = 50, num_return_sequences = 1):

    # Coloca o modelo em modo de previsão
    model.eval()

    # Extrai os tokens do texto de entrada no formato do PyTorch (pt)
    input_ids = tokenizer.encode(prompt, return_tensors = 'pt').to(device)

    # Gera sequências de texto a partir do texto inicial
    generated_sequences = model.generate(input_ids = input_ids,
                                         max_length = max_length,
                                         num_return_sequences = num_return_sequences,
                                         no_repeat_ngram_size = 2,
                                         pad_token_id = tokenizer.eos_token_id,
                                         do_sample = True,
                                         top_k = 50,
                                         top_p = 0.95,
                                         temperature = 0.3)

    return [tokenizer.decode(sequence, skip_special_tokens = True) for sequence in generated_sequences]

No código acima, model.generate() é uma função usada para gerar sequências de texto a partir de um modelo de linguagem. Cada argumento que a função recebe tem um propósito específico:

**input_ids**: Este é o identificador de entrada para o modelo. Geralmente é uma sequência de números que representam o texto de entrada, transformado em tokens por um tokenizer.

**max_length**: Este é o comprimento máximo da sequência de saída. O modelo irá parar de gerar quando atingir este limite.

**num_return_sequences**: Este é o número de sequências independentes a serem retornadas. Cada uma será gerada de um único prompt.

**no_repeat_ngram_size**: Esse é o tamanho do n-grama que não deve ser repetido. Se for 2, por exemplo, o modelo não gerará a mesma sequência de 2 tokens seguidos.

**pad_token_id**: Este é o ID do token que será usado para preenchimento (padding) se a sequência gerada for mais curta que o max_length.

**do_sample**: Se definido como True, a função irá usar amostragem aleatória para gerar o texto, o que significa que poderá haver variações a cada vez que a mesma entrada for fornecida. Se False, a função sempre gerará a mesma saída para uma determinada entrada.

**top_k**: Este parâmetro é usado para limitar a amostragem às K principais probabilidades. O modelo seleciona um dos K tokens mais prováveis para continuar a sequência a cada passo.

**top_p**: Este é o valor de "nucleus sampling" ou "top-p sampling". Ao invés de selecionar as top K probabilidades, o modelo seleciona um conjunto mínimo de tokens que tem uma probabilidade cumulativa de pelo menos p. Isso pode levar a um conjunto de tokens potencialmente menor ou maior do que K, dependendo da distribuição de probabilidade.

**temperature**: Este parâmetro afeta a "aleatoriedade" das escolhas do modelo. Valores mais altos fazem a distribuição de probabilidade mais plana (mais aleatória), enquanto valores mais baixos tornam a distribuição mais aguda (menos aleatória). Por exemplo, uma temperatura de 0.1 fará o modelo quase sempre escolher o token mais provável, enquanto uma temperatura de 1.0 fará a seleção ser mais proporcional às probabilidades dos tokens.

https://huggingface.co/gpt2

In [11]:
# Modelo
PRETRAINED_MODEL = 'gpt2'

In [12]:
# Dispositivo
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# Inicialização do tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(PRETRAINED_MODEL)

In [14]:
# Inicialização do modelo
modelo = GPT2LMHeadModel.from_pretrained(PRETRAINED_MODEL)

In [23]:
# Coloca o modelo no device
modelo.to(DEVICE)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [24]:
# Texto inicial
prompt = 'The advantages of machine learning are'

In [25]:
# Gerando texto
generated_texts = gera_texto(prompt,
                             modelo,
                             tokenizer,
                             DEVICE,
                             max_length = 100,
                             num_return_sequences = 3)

In [26]:
# Imprime o resultado
for idx, generated_text in enumerate(generated_texts):
    print(f'Texto Gerado {idx + 1}:\n{generated_text}\n')

Texto Gerado 1:
The advantages of machine learning are that it can be used to predict the future and to make predictions about the past.

Machine learning is a new field in which researchers have been studying the potential of artificial intelligence for decades. The goal of the field is to understand how the human brain works and how it might be able to learn from experience. In this paper, we present a novel approach to understanding how machine-learning can learn. We show that machine intelligence can predict a future, and that this

Texto Gerado 2:
The advantages of machine learning are that it can be used to predict the future. It can also be applied to other problems, such as predicting the effects of climate change on agriculture.

"We're seeing a lot of new applications of this kind of thing, and it's exciting to see how it evolves," said Dr. David H. Smith, a professor of computer science at the University of California, Berkeley. "We've seen some of the most interesting thing

# Fim